# Import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import json

from sklearn.linear_model import ElasticNet

pd.set_option('display.float_format', lambda x: '%.3f' % x)
sns.set_style('whitegrid')
np.set_printoptions(suppress=True)

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Метрика
def sMdAPE(y_true, y_pred):
    """
    Возвращает значение метрики sMdAPE

    Parameters
    ----------
    y_true : np.array, вектор истинных значений
    y_pred : np.array, вектор предсказанных значений
    """
    difference = abs(y_true - y_pred)
    sum_ = y_true + y_pred
    values = difference/sum_
    median = np.median(values)
    return median

# Load

In [10]:
covid_data = pd.read_csv('data_cv_wt_crd.csv')
covid_dr = covid_data.dropna()

covid = covid_dr[covid_dr.Ковид == 1]
no_covid = covid_dr[covid_dr.Ковид == 0]

# Modeling

In [28]:
features = [
    'Заражений',
    'Максимальная температура, С',
    'Средняя температура, С',
    'Минимальная температура, С',
    'population',
    'latitude_dd',
    'longitude_dd'
]

targets = ['max_3', 'min_3', 'avg_3', 'Count_3']

# Отображение графиков спроса   
def plot_ts(df, preds):
    fig, ax_left = plt.subplots(figsize=(14, 6))
    ax_right = ax_left.twinx()
    ax_left.plot(df.Месяц, df['min_3'], label='min3')
    ax_left.plot(df.Месяц, df['avg_3'], label='avg3')
    ax_left.plot(df.Месяц, df['max_3'], label='max3')
    ax_left.scatter([9, 10], preds, label='predicted', marker='x', color='green')
    ax_right.plot(df.Месяц, df['Заражений'], label='Заболевания', color='red', linestyle='dashed')
    ax_left.legend(loc='upper left')
    ax_right.legend(loc='upper right')
    ax_left.set_title([mnn, subj])
    fig.tight_layout()

# Функция модели линейной регресси
def train_fit_lr(df, tg):
    '''
    df - подаваемый на вход тренировочный датасет
    tg - целевая переменная, которую необходимо предсказать
    '''
    model = ElasticNet(l1_ratio=0)

    x = df[features]
    y = df[tg].values.reshape(-1, 1)
    y = np.log(y)
    
    model.fit(x, y)
    
    return model

# Предобработка сырых данных в тестовые
def test_xy(df, tg):
    '''
    df - подаваемый на вход тестовый датасет
    tg - целевая переменная, которую необходимо предсказать
    '''
    x_test = df[features]
    y_test = df[tg].values.reshape(-1, 1)

    return [x_test, y_test]


mnn_preds = {}
covid_dr = covid
for mnn in covid_dr['МНН'].unique():
    print(mnn)
    subj_preds = {}
    weights_dict = {}
    temp = covid_dr[covid_dr['МНН'] == mnn]

    for subj in temp['Код субъекта'].unique():
        
        weights_target_dict = {}
        preds_target_dict = {}
        
        for target in targets:
            
            target_weights = []
            preds_list = []

            temp_mnn = covid_dr[(covid_dr['Код субъекта'] == subj) & (covid_dr['МНН'] == mnn)]
            temp_mnn.sort_values(by='Месяц_old', inplace=True)
            temp_gr = temp_mnn.groupby('Месяц_old').median().reset_index()

            if ((9 not in temp_gr.Месяц_old.unique()) or (10 not in temp_gr.Месяц_old.unique())) or len(temp_gr.Месяц_old.unique()) < 3:
                # зануление предсказаний в случаях отсутствия тестовых данных
                preds = [0, 0]
                target_weights = [0, 0]
                continue
                
            train = temp_gr[(temp_gr.Месяц_old != 9) & (temp_gr.Месяц_old != 10)]
            test = temp_gr[(temp_gr.Месяц_old == 9) | (temp_gr.Месяц_old == 10)]
            
            model = train_fit_lr(train, target)
            preds = np.exp(model.predict(test_xy(test, target)[0])).tolist()   
            
            target_weights =[model.coef_.tolist(), model.intercept_[0]]
            weights_target_dict[target] = target_weights
            preds_target_dict[target] = preds
                  
        subj_preds[f'{subj}_preds'] = preds_target_dict
        subj_preds[f'{subj}_weights'] = weights_target_dict
    
    mnn_preds[f'{mnn}'] = subj_preds

# Import to JSON

In [ ]:
with open('results.json', 'w', encoding='utf-8') as fp:
    json.dump(mnn_preds, fp, ensure_ascii=False)